# Data Source
https://docs.google.com/spreadsheets/d/1irg60f9qsZOkhp0cwOU7Cy4rJQeyusEUzTNQzhoTYTU/edit?gid=0#gid=0

Let's sanitize this into a nice JSON file to use as a data source for the website.


In [1]:
import { DeviceParser } from "./device-parser/device.parser.ts";
import { EmulationSystemOrder } from "../enums/EmulationSystem.ts";
import { slugify } from "https://deno.land/x/slugify@0.3.0/mod.ts";

slugify.extend({
  '?': 'question-mark'
})

// living data path
const filePath = `results/handhelds.json`;

// parse handhelds html 
const parsedDevicesHandhelds = DeviceParser.parseHandheldsHtml("./files/Handhelds.html");
console.info("parsedDevicesHandhelds: ", parsedDevicesHandhelds.length);

const parsedDevicesOEMs= DeviceParser.parseOEMsHtml("./files/OEM.html");
console.info("parsedDevicesOEMs: ", parsedDevicesOEMs.length);

const allDevices = [...parsedDevicesHandhelds, ...parsedDevicesOEMs];

console.info("all parsed devices: ", allDevices.length);

const devices = allDevices
.filter((device) => device.name.sanitized !== "")
.filter((device) => device.image.originalUrl !== "");

const currentDevices = JSON.parse(new TextDecoder().decode(await Deno.readFile(filePath))) as Device[];

const currentDevicesMap = new Map(currentDevices.map(device => [device.name.raw, device]));

devices.forEach(newDevice => {
  const currentDevice = currentDevicesMap.get(newDevice.name.raw);

  if (!currentDevice) {
    console.info(`New device added: ${newDevice.name.raw}`);
  }
});

parsedDevicesHandhelds:  439
parsedDevicesOEMs:  24
all parsed devices:  463
New device added: K56
New device added: K560


In [2]:

// marshall into JSON and into bytes
const devicesJson = JSON.stringify(devices, null, 2);
const devicesJsonBytes = new TextEncoder().encode(devicesJson);

In [3]:
// place source/handhelds.json

await Deno.mkdir('results', { recursive: true });
await Deno.writeFile(filePath, devicesJsonBytes);
